  # Created Profile embeddings using trained autoencoder

# Setup

In [1]:
from IPython.display import display, HTML, Image
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload  
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
import sys
import csv
import time
import types
import copy
import pprint
import logging
from datetime import datetime
for p in ['./src','../..']:
    if p not in sys.path:
        print(f"insert {p}")
        sys.path.insert(0, p)
print(sys.path)
from typing import Dict, List, Tuple

import tqdm
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import scipy
from scipy.sparse import csr_matrix
import torch
from tqdm import tqdm
import torch.nn.functional as F
from torchinfo import summary
import wandb

pp = pprint.PrettyPrinter(indent=4)
pd.options.display.width = 132
torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=150, profile=None, sci_mode=None)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')

os.environ["WANDB_NOTEBOOK_NAME"] = "AE-CreateEmbedding.ipynb"
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
torch.set_num_threads(3)  ## <--- limit to ~ 2 CPUs
torch.get_num_threads()

insert ./src
insert ../..
['../..', './src', '/home/kevin/miniforge3/envs/ptsnnl/lib/python311.zip', '/home/kevin/miniforge3/envs/ptsnnl/lib/python3.11', '/home/kevin/miniforge3/envs/ptsnnl/lib/python3.11/lib-dynload', '', '/home/kevin/miniforge3/envs/ptsnnl/lib/python3.11/site-packages', '/home/kevin/miniforge3/envs/ptsnnl/lib/python3.11/site-packages/cytominer_eval-0.1-py3.11.egg']


3

In [5]:
from KevinsRoutines.utils.utils_general import list_namespace
from KevinsRoutines.utils.utils_gpu import get_device, set_device

import snnl.utils as utils
from snnl.utils import parse_args, load_configuration, set_global_seed
from snnl.utils import display_model_summary, define_autoencoder_model


In [6]:
timestamp = datetime.now().strftime('%Y_%m_%d_%H:%M:%S')
logger = logging.getLogger(__name__) 
logLevel = os.environ.get('LOG_LEVEL', 'INFO').upper()
FORMAT = '%(asctime)s - %(name)s - %(levelname)s: - %(message)s'
logging.basicConfig(level="INFO", format= FORMAT)

logger.info(f" Excution started : {timestamp} ")
logger.info(f" Pytorch version  : {torch.__version__}")
logger.info(f" Scipy version    : {scipy.__version__}  \t\t Numpy version : {np.__version__}")
logger.info(f" WandB version    : {wandb.__version__}  \t\t Pandas version: {pd.__version__}  ")


2024-10-02 17:21:01,200 - __main__ - INFO: -  Excution started : 2024_10_02_17:21:01 
2024-10-02 17:21:01,201 - __main__ - INFO: -  Pytorch version  : 2.2.2
2024-10-02 17:21:01,202 - __main__ - INFO: -  Scipy version    : 1.12.0  		 Numpy version : 1.26.4
2024-10-02 17:21:01,203 - __main__ - INFO: -  WandB version    : 0.17.4  		 Pandas version: 2.2.1  


### main(args)

In [22]:
RUNMODE       = "baseline"
# RUNMODE       = "snnl"
WANDB_ACTIVE  = False
LATENT_DIM    = 150
HIDDEN_1      = 512
EPOCHS        = 5
GPU_ID        = 2
COMPOUNDS_PER_BATCH = 600
ADAM_WEIGHT_DECAY = 0.001
SNNL_FACTOR   = 3.0

# CONFIGURATION = "./hyperparameters/ae_baseline_150_512_cpb.yaml"
# CONFIGURATION   = "./hyperparameters/ae_sn_150_512_cpb.yaml"

In [23]:
cli_args = f" --runmode            {RUNMODE} " \
            f" --configuration      hyperparameters/ae_{RUNMODE}_{LATENT_DIM:03d}_{HIDDEN_1}_cpb.yaml" \
            f" --epochs             {EPOCHS} " \
            f" --seed               4321" \
            f" --cpb                {COMPOUNDS_PER_BATCH}" \
            f" --adam_wd            {ADAM_WEIGHT_DECAY}"\
            f" --snnl_factor        {SNNL_FACTOR}"\
            f" --gpu_id             {GPU_ID}  "  \
            f" "
cli_args = utils.parse_args(cli_args.split())
cli_args

args = utils.load_configuration(cli_args)
args


Namespace(configuration='hyperparameters/ae_baseline_150_512_cpb.yaml', ckpt=None, cpb=600, exp_title=None, epochs=5, gpu_id=2, learning_rate=None, exp_id=None, runmode='baseline', random_seed=4321, use_prim_optimizer=True, use_temp_optimizer=False, use_annealing=False, anneal_patience=15, use_single_loss=False, temperature=None, adam_weight_decay=0.001, loss_factor=None, snnl_factor=3.0, temperatureLR=None, WANDB_ACTIVE=False)

2024-10-02 17:50:38,639 - snnl.utils.utils_ptsnnl - INFO: -  command line param configuration             : [hyperparameters/ae_baseline_150_512_cpb.yaml]
2024-10-02 17:50:38,639 - snnl.utils.utils_ptsnnl - INFO: -  command line param ckpt                      : [None]
2024-10-02 17:50:38,640 - snnl.utils.utils_ptsnnl - INFO: -  command line param cpb                       : [600]
2024-10-02 17:50:38,641 - snnl.utils.utils_ptsnnl - INFO: -  command line param exp_title                 : [None]
2024-10-02 17:50:38,642 - snnl.utils.utils_ptsnnl - INFO: -  command line param epochs                    : [5]
2024-10-02 17:50:38,642 - snnl.utils.utils_ptsnnl - INFO: -  command line param gpu_id                    : [2]
2024-10-02 17:50:38,643 - snnl.utils.utils_ptsnnl - INFO: -  command line param learning_rate             : [None]
2024-10-02 17:50:38,644 - snnl.utils.utils_ptsnnl - INFO: -  command line param exp_id                    : [None]
2024-10-02 17:50:38,644 - snnl.utils.utils_ptsn

namespace(project_name='CellPainting_Profiles',
          exp_id=None,
          exp_name_pfx=None,
          exp_name='AE_20241002_1750',
          exp_date='20241002_1750',
          exp_title='scpb600_150Ltnt_512',
          exp_description='AE baseline - SnglOpt, latent: 150, hidden_1: 512, cpb: 600',
          exp_folder=None,
          folder_sfx=None,
          random_seed=4321,
          units=[['linear', 1471, 1024],
                 ['relu', 0, 0],
                 ['linear', 1024, 512],
                 ['relu', 0, 0],
                 ['linear', 512, 150],
                 ['linear', 150, 512],
                 ['relu', 0, 0],
                 ['linear', 512, 1024],
                 ['relu', 0, 0],
                 ['linear', 1024, 1471]],
          input_shape=1471,
          hidden_1=512,
          code_units=150,
          embedding_layer=4,
          show_every=1,
          save_every=100,
          use_prim_optimizer=True,
          use_prim_scheduler=True,
          l

In [24]:
set_global_seed(args.random_seed)
if args.ckpt is not None:
    if os.path.exists(os.path.join('ckpts', args.ckpt)):
        logger.info(f"Checkpoint {args.ckpt} found")
        logger.info(f"Resuming training using checkpoint: {args.ckpt}")
    else:
        logger.error(f"*** Checkpoint {args.ckpt} not found *** \n")
        raise ValueError(f"\n *** Checkpoint DOESNT EXIST *** \n")

if args.gpu_id is not None:
    # _ = get_device(verbose=True)
    # print(f"Current device is : {current_device}")
    args.current_device = set_device(args.gpu_id)


 Switched to: "cuda:2"   Device Name: NVIDIA TITAN Xp               


# Define Dataloaders

In [61]:
# args.cellpainting_args['validation_path']

args.cellpainting_args['training_path'] = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_all.csv'
args.cellpainting_args['train_start'] = 0
args.cellpainting_args['train_end'] = 312_000    # 346542   # 312,000 + 34,542 = 346542

args.cellpainting_args['validation_path'] = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training_sub_val.csv'
args.cellpainting_args['val_start'] = 0
args.cellpainting_args['val_end'] = 21_600

args.cellpainting_args['test_path'] = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training_sub_test.csv'
args.cellpainting_args['test_start'] = 0 
args.cellpainting_args['test_end'] = 12_600

# args.cellpainting_args['training_path'] = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training.csv'
# args.cellpainting_args['train_start'] = 0
# args.cellpainting_args['train_end'] = 399_000  # 312_000

# args.cellpainting_args['test_path'] = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_test.csv'
# args.cellpainting_args['test_start'] = 0 
# args.cellpainting_args['test_end'] = 34_542

args.cellpainting_args

{'sample_size': 3,
 'batch_size': 1,
 'compounds_per_batch': 600,
 'training_path': '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_all.csv',
 'validation_path': '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training_sub_val.csv',
 'test_path': '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_training_sub_test.csv',
 'train_start': 0,
 'train_end': 312000,
 'val_start': 0,
 'val_end': 21600,
 'test_start': 0,
 'test_end': 12600,
 'chunksize': None,
 'conversions': None,
 'iterator': True}

In [62]:
data_loader = utils.build_dataloaders(args, data = ['train', 'val', 'test'])
data_loader.keys()

2024-10-02 18:58:47,845 - root - INFO: -  load cellpainting
2024-10-02 18:58:47,846 - snnl.utils.dataloader - INFO: -  Building CellPantingDataset for train
2024-10-02 18:58:47,847 - snnl.utils.dataloader - INFO: -  filename:  /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_profiles/3sample_profiles_1482_HashOrder_all.csv
2024-10-02 18:58:47,847 - snnl.utils.dataloader - INFO: -  type    :  train
2024-10-02 18:58:47,848 - snnl.utils.dataloader - INFO: -  start   :  0
2024-10-02 18:58:47,849 - snnl.utils.dataloader - INFO: -  end     :  312000
2024-10-02 18:58:47,849 - snnl.utils.dataloader - INFO: -  numrows :  312000
2024-10-02 18:58:47,850 - snnl.utils.dataloader - INFO: -  names   :  None     usecols :  None
2024-10-02 18:58:47,851 - snnl.utils.dataloader - INFO: -  batch_size  :  1
2024-10-02 18:58:47,851 - snnl.utils.dataloader - INFO: -  sample_size :  3
2024-10-02 18:58:47,852 - snnl.utils.dataloader - INFO: -  compounds_per_batch :  600
2024-10-02 18:58

 load cellpainting
 Dataset size: 312000   rows per batch: 1800  tpsa_threshold: 100
 Dataset size: 21600   rows per batch: 1800  tpsa_threshold: 100
 Dataset size: 12600   rows per batch: 1800  tpsa_threshold: 100


dict_keys(['train', 'val', 'test'])

In [63]:
# for self.batch_count, (batch_features, batch_labels, _, _, _, _) in enumerate(data_loader):
# for idx, batch in enumerate(data_loader['train']):
#     print(batch[0].shape[0], batch[1].sum())
#     # display_cellpainting_batch(idx, batch)
#     if idx == 1:
#         break

In [64]:
# # -----------------------------------------
# #  Count pos/neg labels in each dataset
# # -----------------------------------------
# MINIBATCH_SIZE = args.cellpainting_args['sample_size'] * args.cellpainting_args['compounds_per_batch']
# print(f" Minibatch size : {MINIBATCH_SIZE} \n") 

# for datatype in ['train', 'val','test']:
#     minibatches = len(data_loader[datatype]) // MINIBATCH_SIZE
#     ttl_rows = 0
#     ttl_pos_labels = 0 
#     with tqdm(enumerate(data_loader[datatype]), initial=0, total = minibatches, position=0, file=sys.stdout,
#               leave= False, 
#               desc=f" Count labels ") as t_warmup:
#         for batch_count, (batch_features, batch_labels, _, _, _, _) in t_warmup:
#             ttl_rows += batch_labels.shape[0]
#             ttl_pos_labels += batch_labels.sum()
#     ttl_neg_labels = ttl_rows - ttl_pos_labels
#     ttl = f"\n Dataset: {datatype} -  len of {datatype} data loader: {len(data_loader[datatype])}   number of batches: {minibatches}"
#     print(ttl)
#     print('-'*len(ttl))
#     print(f" total rows     : {ttl_rows:7d}")
#     print(f" total pos rows : {ttl_pos_labels:7.0f} - {ttl_pos_labels*100.0/ttl_rows:5.2f}%")
#     print(f" total neg rows : {ttl_neg_labels:7.0f} - {ttl_neg_labels*100.0/ttl_rows:5.2f}%")
#     print()

     Minibatch size : 1800 
                                                                                                                                                                               
     Dataset: train
    ------------------------------
     total rows     : 312000
     total pos rows :   37164 - 11.91%
     total neg rows :  274836 - 88.09%

     Dataset: val -  len of val data loader: 21600   number of batches: 12
    -----------------------------------------------------------------------
     total rows     :   21600
     total pos rows :    2532 - 11.72%
     total neg rows :   19068 - 88.28%
    
                                                                                                                                                                                                                  
     Dataset: test -  len of test data loader: 12600   number of batches: 7
    ------------------------------------------------------------------------
     total rows     :   12600
     total pos rows :    1431 - 11.36%
     total neg rows :   11169 - 88.64%     


# Define Autoencoder Model

In [65]:
print(f"   Units             {args.units}")
print(f"   Latent dim        {args.code_units}")
print(f"   loss_factor       {args.loss_factor}")
print(f"   snnl_factor       {args.snnl_factor}")
print(f"   temperature       {args.temperature}")
print(f"   learning_rate     {args.learning_rate}")
print(f"   temperatureLR:    {args.temperatureLR}")

# list_namespace(args)

   Units             [['linear', 1471, 1024], ['relu', 0, 0], ['linear', 1024, 512], ['relu', 0, 0], ['linear', 512, 150], ['linear', 150, 512], ['relu', 0, 0], ['linear', 512, 1024], ['relu', 0, 0], ['linear', 1024, 1471]]
   Latent dim        150
   loss_factor       1.0
   snnl_factor       3.0
   temperature       0.0
   learning_rate     0.001
   temperatureLR:    0.0


In [66]:
args.runmode
model = define_autoencoder_model(args, verbose=True)
print(args.current_device, model.device)

'baseline'

2024-10-02 18:58:51,176 - snnl.utils.utils_notebook - INFO: - Defining model in baseline mode
2024-10-02 18:58:51,177 - snnl.models.autoencoder - INFO: -     layer pair:    0  type:linear           input:   1471  output:   1024    weights: [1024, 1471]   
2024-10-02 18:58:51,189 - snnl.models.autoencoder - INFO: -     layer pair:    1  type:relu             input:      0  output:      0    weights: [0, 0]   
2024-10-02 18:58:51,190 - snnl.models.autoencoder - INFO: -     layer pair:    2  type:linear           input:   1024  output:    512    weights: [512, 1024]   
2024-10-02 18:58:51,195 - snnl.models.autoencoder - INFO: -     layer pair:    3  type:relu             input:      0  output:      0    weights: [0, 0]   
2024-10-02 18:58:51,195 - snnl.models.autoencoder - INFO: -     layer pair:    4  type:linear           input:    512  output:    150    weights: [150, 512]   
2024-10-02 18:58:51,197 - snnl.models.autoencoder - INFO: -     layer pair:    5  type:linear           input: 

 EMBEDDING LAYER: 4
 Device cuda:2 will be used

------------------------------------------------------------
 Building Base Model from NOTEBOOK
------------------------------------------------------------
    Model_init()_    -- mode:              autoencoding
    Model_init()_    -- Unsupervised :     True
    Model_init()_    -- Support for unsupervised training  in 'autoencoding' mode is True
    Model_init()_    -- Criterion:         MSELoss()
    Model_init()_    -- use_snnl :         False
    Model_init()_    -- temperature :      0
    Model_init()_    -- temperature LR:    0.0

------------------------------------------------------------
 Building Autoencoder from NOTEBOOK
------------------------------------------------------------
setup_prim_optimizer()
    AE init() -- mode               : autoencoding
    AE init() -- unsupervised       : True
    AE init() -- layer_types        : ['linear', 'relu', 'linear', 'relu', 'linear', 'linear', 'relu', 'linear', 'relu', 'linear']

In [67]:
# display_model_summary(model)

# list_namespace(args)

# Model checkpoint files

In [68]:
args.load_checkpoint = utils.load_checkpoint_v5
args.save_checkpoint = utils.save_checkpoint_v5
args.ckpt

In [69]:
ex_suffix = '*'

if args.runmode == 'snnl':
    # ex_runmode = 'snnl'
    # ex_datetime = "20240709_2235"
    # ex_datetime = "20240829_1712"
    # ex_datetime = "20240906_2201"   # <-- CPB: 600, Latent 150, WD = 0.001, SNN Factor 3
    # ex_datetime = "20240917_2004"   # <-- CPB: 600, Latent 250, WD = 0.001, SNN Factor 3
    # ex_datetime = "20240924_0146"   # <-- CPB: 600, Latent 250, WD = 0.001, SNN Factor 30
    ex_indicator = 'd'
else:
    ex_runmode = 'base'
    # ex_datetime = "20240917_2017"   # <-- CPB: 600, Latent 250, WD = 0.001
    ex_datetime = "20240923_1943"   # <-- CPB: 600, Latent 150, WD = 0.001
    ex_indicator = 's'


In [70]:
# filename = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_{epochs:03d}_cpb_{ex_cpb}_factor_{ex_factor:d}.pt"
# file_pattern = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_*_cpb_{ex_cpb}_factor_{ex_factor:d}.pt"
# filename = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_{epochs:03d}_cpb_{ex_cpb}_factor_{ex_factor:d}.pt"
# file_pattern = f"{model.name}_{ex_runmode}_{ex_date}_{ex_title}_ep_*.pt"
# print(filename)
# print(file_pattern)

filename = f"{model.name}_{ex_runmode}_{ex_indicator}cpb{COMPOUNDS_PER_BATCH}_{LATENT_DIM}Ltnt_{HIDDEN_1}_{ex_datetime}.pt"
file_pattern = f"{model.name}_{ex_runmode}_{ex_indicator}cpb{COMPOUNDS_PER_BATCH}_{LATENT_DIM}Ltnt_{HIDDEN_1}_{ex_datetime}{ex_suffix}.pt"
print(filename)
print(file_pattern)
print(args.ckpt)

AE_base_scpb600_150Ltnt_512_20240923_1943.pt
AE_base_scpb600_150Ltnt_512_20240923_1943*.pt
None


In [71]:
import glob
ckpt_filelist = glob.glob(file_pattern,root_dir = './ckpts')
ckpt_filelist = sorted(ckpt_filelist)
for i, f in enumerate(ckpt_filelist):
    print(f" {i:3d} :  {f}")
# epochlist =sorted([int(x[-6:-3]) for x in ckpt_filelist])
# epochlist

   0 :  AE_base_scpb600_150Ltnt_512_20240923_1943_BEST.pt
   1 :  AE_base_scpb600_150Ltnt_512_20240923_1943_LAST_ep_600.pt
   2 :  AE_base_scpb600_150Ltnt_512_20240923_1943_ep_100.pt
   3 :  AE_base_scpb600_150Ltnt_512_20240923_1943_ep_200.pt
   4 :  AE_base_scpb600_150Ltnt_512_20240923_1943_ep_300.pt
   5 :  AE_base_scpb600_150Ltnt_512_20240923_1943_ep_400.pt
   6 :  AE_base_scpb600_150Ltnt_512_20240923_1943_ep_500.pt
   7 :  AE_base_scpb600_150Ltnt_512_20240923_1943_ep_600.pt


### Test loading a checkpoint

In [72]:
ckpt_file = ckpt_filelist[0]
ckpt_file

'AE_base_scpb600_150Ltnt_512_20240923_1943_BEST.pt'

In [73]:
model_copy, last_epoch = args.load_checkpoint(model, ckpt_file, verbose = False)
model_copy.train()
model_copy.device = args.current_device
model_copy = model_copy .cuda(device=args.current_device)
logging.info(f" Loaded Model device {model_copy.device} -  Last completed epoch : {last_epoch}")

args.current_device, model.device, model_copy.device
del model_copy

2024-10-02 18:59:03,433 - root - INFO: - Load model checkpoint from  AE_base_scpb600_150Ltnt_512_20240923_1943_BEST.pt
2024-10-02 18:59:03,473 - snnl.utils.utils_ptsnnl - INFO: -  Loaded from checkpoint AE_base_scpb600_150Ltnt_512_20240923_1943_BEST.pt successfully. last epoch on checkpoint: 567
2024-10-02 18:59:03,474 - snnl.utils.utils_ptsnnl - INFO: -  Model best training metric   : 0.664312 - epoch: 553
2024-10-02 18:59:03,475 - snnl.utils.utils_ptsnnl - INFO: -  Model best validation metric : 0.696222 - epoch: 567


Autoencoder(
  (primary_criterion): MSELoss()
  (layers): ModuleList(
    (0): Linear(in_features=1471, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=150, bias=True)
    (5): Linear(in_features=150, out_features=512, bias=True)
    (6): ReLU()
    (7): Linear(in_features=512, out_features=1024, bias=True)
    (8): ReLU()
    (9): Linear(in_features=1024, out_features=1471, bias=True)
  )
)

2024-10-02 18:59:03,479 - root - INFO: -  Loaded Model device cuda:2 -  Last completed epoch : 567


('cuda:2', 'cuda:2', 'cuda:2')

# Embedding Output file 

In [74]:
embedding_file_cols = ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_Hash', 'Metadata_Bin', 'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA', 'Metadata_Permiation']
embedding_file_cols += [f'Feature_{x:03d}' for x in range(LATENT_DIM)]
output_file_cols = len(embedding_file_cols)
print(output_file_cols)
OUTPUT_PATH = f"/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/"

161


In [80]:
ae_datetime = ex_datetime     # Autoencoder training - WD = 0.001, SNN Factor 3, CPB = 600
ae_runmode = ex_runmode
# ae_ckpttype = "LAST"
ae_ckpttype = "BEST"
# ae_ckpttype = "ep100"
 

In [81]:
OUTPUT_FILE = f"3smpl_prfl_embedding_{output_file_cols}_HashOrder_{ae_runmode}_{ae_datetime}_{ae_ckpttype}_{{datatype}}.csv"
OUTPUT_FILE = os.path.join(OUTPUT_PATH, OUTPUT_FILE)

for data in ['train', 'val', 'test']:
    print(f" {data} output file: \t {OUTPUT_FILE.format(datatype = data)}")


 train output file: 	 /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_base_20240923_1943_BEST_train.csv
 val output file: 	 /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_base_20240923_1943_BEST_val.csv
 test output file: 	 /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_base_20240923_1943_BEST_test.csv


# Load specific checkpoint

In [82]:
ckpt_file = ckpt_filelist[0]
ckpt_file

'AE_base_scpb600_150Ltnt_512_20240923_1943_BEST.pt'

In [83]:
model, last_epoch = utils.load_model_from_ckpt(model, args, filename = ckpt_file, cuda_device = model.device, verbose = False)
if model is None:
    print(f"model is none")
else:
    print(f"model loaded - last epoch is {last_epoch}")

2024-10-02 19:05:34,037 - root - INFO: - Load model checkpoint from  AE_base_scpb600_150Ltnt_512_20240923_1943_BEST.pt
2024-10-02 19:05:34,077 - snnl.utils.utils_ptsnnl - INFO: -  Loaded from checkpoint AE_base_scpb600_150Ltnt_512_20240923_1943_BEST.pt successfully. last epoch on checkpoint: 567
2024-10-02 19:05:34,078 - snnl.utils.utils_ptsnnl - INFO: -  Model best training metric   : 0.664312 - epoch: 553
2024-10-02 19:05:34,079 - snnl.utils.utils_ptsnnl - INFO: -  Model best validation metric : 0.696222 - epoch: 567
2024-10-02 19:05:34,095 - snnl.utils.utils_ptsnnl - INFO: -  Loaded model device     : cuda:2


model loaded - last epoch is 567


# Get latent representations of Training and Test data

In [84]:

for datatype, file_suffix in [('train', 'train'), ('val', 'train_sub_val'), ('test', 'train_sub_test')]:
    print(f"****************  {datatype} - get latent representation ***************")
    write_header = True
    mdl_outputs = utils.get_latent_representation(model, data_loader[datatype], embedding_layer = 4, verbose = False)

    print(f"****************  {datatype} - convert to dataframe  ***************")
    df_output = pd.DataFrame.from_records(mdl_outputs, columns = embedding_file_cols )

    print(f"****************  {datatype} - write to csv file : {OUTPUT_FILE.format(datatype=datatype)} ***************")
    df_output.to_csv(os.path.join(OUTPUT_PATH, OUTPUT_FILE.format(datatype=file_suffix)), mode = 'a', index = False, header = write_header)
    write_header = False


****************  train - get latent representation ***************
 embedding layer: 4 - model.layer: in: 512 out: 150
(1800, 161) (1800, 161)
(1800, 161) (3600, 161)
(1800, 161) (5400, 161)
(1800, 161) (7200, 161)
(1800, 161) (9000, 161)
(1800, 161) (10800, 161)
(1800, 161) (12600, 161)
(1800, 161) (14400, 161)
(1800, 161) (16200, 161)
(1800, 161) (18000, 161)
(1800, 161) (19800, 161)
(1800, 161) (21600, 161)
(1800, 161) (23400, 161)
(1800, 161) (25200, 161)
(1800, 161) (27000, 161)
(1800, 161) (28800, 161)
(1800, 161) (30600, 161)
(1800, 161) (32400, 161)
(1800, 161) (34200, 161)
(1800, 161) (36000, 161)
(1800, 161) (37800, 161)
(1800, 161) (39600, 161)
(1800, 161) (41400, 161)
(1800, 161) (43200, 161)
(1800, 161) (45000, 161)
(1800, 161) (46800, 161)
(1800, 161) (48600, 161)
(1800, 161) (50400, 161)
(1800, 161) (52200, 161)
(1800, 161) (54000, 161)
(1800, 161) (55800, 161)
(1800, 161) (57600, 161)
(1800, 161) (59400, 161)
(1800, 161) (61200, 161)
(1800, 161) (63000, 161)
(1800, 161

In [4]:
for datatype, file_suffix in [('train', 'train'), ('val', 'train_sub_val'), ('test', 'train_sub_test')]:
    print(datatype, file_suffix)

train train
val train_sub_val
test train_sub_test


# Misc - Read Embedding file just created 

In [ ]:
metadata_cols = ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022','Metadata_Hash','Metadata_Bin', 'Metadata_TPSA', 'Metadata_lnTPSA', 'Metadata_log10TPSA', 'Metadata_Permiation']

In [85]:
INPUT_FILE = os.path.join(OUTPUT_PATH, OUTPUT_FILE.format(datatype='train'))
# INPUT_FILE = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240906_2201_LAST_all.csv'
# INPUT_FILE = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240906_2201_LAST_train.csv'
INPUT_FILE = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240829_1712_LAST_train.csv'
INPUT_FILE


'/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240829_1712_LAST_train.csv'

In [86]:
# df_input = pd.read_csv(INPUT_FILE,usecols=['Metadata_Source'])
df_train = pd.read_csv(INPUT_FILE,usecols=metadata_cols)

In [87]:
df_train.shape
df_train.info()

(312000, 11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312000 entries, 0 to 311999
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Metadata_Source      312000 non-null  object 
 1   Metadata_Batch       312000 non-null  object 
 2   Metadata_Plate       312000 non-null  object 
 3   Metadata_Well        312000 non-null  object 
 4   Metadata_JCP2022     312000 non-null  object 
 5   Metadata_Hash        312000 non-null  int64  
 6   Metadata_Bin         312000 non-null  int64  
 7   Metadata_TPSA        312000 non-null  float64
 8   Metadata_lnTPSA      312000 non-null  float64
 9   Metadata_log10TPSA   312000 non-null  float64
 10  Metadata_Permiation  312000 non-null  float64
dtypes: float64(4), int64(2), object(5)
memory usage: 26.2+ MB


In [88]:
df_train

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,Metadata_Permiation
0,source_1,Batch5_20221030,UL001793,M26,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,0.0
1,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143628,M04,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,0.0
2,source_3,CP_31_all_Phenix1,B040603b,G13,JCP2022_006020,-9223347314827979542,10,71.18,4.265212,1.852358,0.0
3,source_6,p211004CPU2OS48hw384exp031JUMP,110000295562,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,0.0
4,source_3,CP_32_all_Phenix1,B40003aW,M08,JCP2022_058905,-9223229703754907651,13,58.65,4.071588,1.768268,0.0
...,...,...,...,...,...,...,...,...,...,...,...
311995,source_6,p211004CPU2OS48hw384exp031JUMP,110000296294,N12,JCP2022_105575,7406061371488308302,14,50.19,3.915816,1.700617,0.0
311996,source_2,20210823_Batch_10,1086292051,L21,JCP2022_105575,7406061371488308302,14,50.19,3.915816,1.700617,0.0
311997,source_10,2021_06_22_U2OS_48_hr_run8,Dest210622-143006,A07,JCP2022_095379,7406311667471038063,15,45.98,3.828207,1.662569,0.0
311998,source_6,p210928CPU2OS48hw384exp030JUMP,110000296370,A15,JCP2022_095379,7406311667471038063,15,45.98,3.828207,1.662569,0.0


In [71]:
y_label = df_train.Metadata_Permiation.astype(np.int8)
bin_cnt = np.bincount(y_label)
bin_cnt
bin_cnt.sum()

array([305214,  41328])

346542

In [54]:
312000+34542

346542

### test file

In [89]:
INPUT_FILE = os.path.join(OUTPUT_PATH, OUTPUT_FILE.format(datatype='test'))
# INPUT_FILE = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240906_2201_LAST_all.csv'
# INPUT_FILE = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240906_2201_LAST_test.csv'
INPUT_FILE = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240829_1712_LAST_test.csv'
INPUT_FILE


'/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240829_1712_LAST_test.csv'

In [90]:
df_test= pd.read_csv(INPUT_FILE,usecols=['Metadata_Source'])
# df_test = pd.read_csv(INPUT_FILE,usecols=metadata_cols)
# df_test= pd.read_csv(INPUT_FILE,num)

In [91]:
df_test.info()
df_test.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34542 entries, 0 to 34541
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Metadata_Source  34542 non-null  object
dtypes: object(1)
memory usage: 270.0+ KB


(34542, 1)

In [43]:
df_test

,Metadata_Source,Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_JCP2022,Metadata_Hash,Metadata_Bin,Metadata_TPSA,Metadata_lnTPSA,Metadata_log10TPSA,Metadata_Permiation
0,source_11,Batch2,EC000046,K04,JCP2022_009278,7406361908543180200,8,62.78,4.139637,1.797821,0.0
1,source_11,Batch2,EC000063,K04,JCP2022_009278,7406361908543180200,8,62.78,4.139637,1.797821,0.0
2,source_9,20211013-Run14,GR00003281,J15,JCP2022_009278,7406361908543180200,8,62.78,4.139637,1.797821,0.0
3,source_5,JUMPCPE-20210908-Run28_20210909_072022,AETJUM107,J10,JCP2022_045457,7406621140030190348,12,33.20,3.502550,1.521138,0.0
4,source_11,Batch3,EC000139,G04,JCP2022_045457,7406621140030190348,12,33.20,3.502550,1.521138,0.0
...,...,...,...,...,...,...,...,...,...,...,...
34537,source_9,20211103-Run16,GR00004412,B19,JCP2022_040466,9222528730267146426,10,98.00,4.584967,1.991226,0.0
34538,source_9,20211103-Run16,GR00004410,R43,JCP2022_040466,9222528730267146426,10,98.00,4.584967,1.991226,0.0
34539,source_6,p211109CPU2OS48hw384exp035JUMP,110000296165,N13,JCP2022_102583,9223213570900537648,0,56.14,4.027849,1.749272,0.0
34540,source_1,Batch4_20221012,UL000093,AB26,JCP2022_102583,9223213570900537648,0,56.14,4.027849,1.749272,0.0


In [73]:
y_label = df_test.Metadata_Permiation.astype(np.int8)
bin_cnt = np.bincount(y_label)
bin_cnt

NameError: name 'df_test' is not defined

In [66]:
INPUT_FILE = os.path.join(OUTPUT_PATH, OUTPUT_FILE.format(datatype='train'))
INPUT_FILE

'/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240906_2201_LAST_train.csv'

In [72]:
del df_train, df_input

NameError: name 'df_input' is not defined

### Split training file to sub_val and sub_test

In [98]:
INPUT_FILE = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240829_1712_LAST_train.csv'
# INPUT_FILE = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240906_2201_LAST_train.csv'
INPUT_FILE
VAL_OUTPUT_FILE = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240829_1712_LAST_train_sub_val.csv'
TEST_OUTPUT_FILE = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240829_1712_LAST_train_sub_test.csv'
# VAL_OUTPUT_FILE = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240906_2201_LAST_train_sub_val.csv'
# TEST_OUTPUT_FILE = '/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240906_2201_LAST_train_sub_test.csv'
VAL_OUTPUT_FILE
TEST_OUTPUT_FILE

'/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240829_1712_LAST_train.csv'

'/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240829_1712_LAST_train_sub_val.csv'

'/home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240829_1712_LAST_train_sub_test.csv'

In [99]:
df_validation = pd.read_csv(INPUT_FILE, header = 0,
                            skiprows = 277_200,
                            nrows = 21_600,
                            )
df_validation.shape
df_validation.info()
df_validation.head(6)
df_validation.tail(6)

(21600, 161)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21600 entries, 0 to 21599
Columns: 161 entries, source_5 to -0.0001686492469161749
dtypes: float64(154), int64(2), object(5)
memory usage: 26.5+ MB


,source_5,JUMPCPE-20210706-Run06_20210706_235916,APTJUM213,I14,JCP2022_100922,5548998150937919747,3,79.96,4.381526509632199,1.90287278544608,...,-0.0010494652669876814,0.0003347290330566466,-0.00024723977549001575,-0.00046131041017360985,0.0009702356182970107,0.0003158403851557523,-7.763417670503259e-06,0.0003491034440230578,0.0011775164166465402,-0.0001686492469161749
0,source_2,20210607_Batch_2,1053601824,N20,JCP2022_101506,5549160564573729245,13,73.85,4.302036,1.868350,...,-0.417622,-0.490450,-0.365111,-0.479673,0.227926,-0.149534,-0.574326,-0.470133,0.031690,0.531656
1,source_10,2021_06_15_U2OS_48_hr_run6,Dest210615-151512,N20,JCP2022_101506,5549160564573729245,13,73.85,4.302036,1.868350,...,-0.582942,-1.235308,-0.323901,-0.867346,-0.238169,-0.048559,-0.750650,-0.558930,0.352158,0.835471
2,source_8,J3,A1170508,G11,JCP2022_101506,5549160564573729245,13,73.85,4.302036,1.868350,...,-0.711429,-1.393002,-0.190025,-1.032771,-0.265787,-0.157114,-0.930913,-0.657756,0.221662,1.170505
3,source_9,20210915-Run10,GR00003318,W40,JCP2022_010664,5549241656037143871,15,74.68,4.313212,1.873204,...,0.000159,-0.000057,0.000103,0.000147,-0.000103,0.000008,-0.000083,-0.000057,0.000030,-0.000029
4,source_9,20210915-Run10,GR00003316,G16,JCP2022_010664,5549241656037143871,15,74.68,4.313212,1.873204,...,0.000405,-0.000068,0.000208,0.000245,-0.000199,-0.000006,-0.000102,-0.000167,-0.000116,-0.000017
5,source_5,JUMPCPE-20210730-Run15_20210801_205335,ATSJUM216,G16,JCP2022_010664,5549241656037143871,15,74.68,4.313212,1.873204,...,0.000616,-0.000608,-0.000324,0.000135,-0.001111,-0.000401,0.000298,-0.000094,-0.001228,0.000248


,source_5,JUMPCPE-20210706-Run06_20210706_235916,APTJUM213,I14,JCP2022_100922,5548998150937919747,3,79.96,4.381526509632199,1.90287278544608,...,-0.0010494652669876814,0.0003347290330566466,-0.00024723977549001575,-0.00046131041017360985,0.0009702356182970107,0.0003158403851557523,-7.763417670503259e-06,0.0003491034440230578,0.0011775164166465402,-0.0001686492469161749
21594,source_2,20210830_Batch_11,1086290668,N21,JCP2022_094244,6703113441807921775,15,80.04,4.382527,1.903307,...,-2.477005e-05,0.000014,-5.094483e-06,-0.000011,-0.000014,0.000004,-2.806945e-05,2.790424e-05,0.000070,-0.000008
21595,source_8,J3,A1170495,G22,JCP2022_094244,6703113441807921775,15,80.04,4.382527,1.903307,...,-1.245223e-05,0.000003,1.573494e-05,0.000014,-0.000047,-0.000023,-2.634252e-06,1.623124e-05,0.000069,-0.000015
21596,source_6,p211109CPU2OS48hw384exp035JUMP,110000295493,G22,JCP2022_094244,6703113441807921775,15,80.04,4.382527,1.903307,...,-4.397180e-05,0.000043,-9.750520e-07,0.000001,-0.000022,0.000018,-2.945206e-05,3.523116e-05,0.000050,-0.000014
21597,source_9,20211103-Run16,GR00004419,R42,JCP2022_040230,6703257770840297119,15,79.29,4.373112,1.899218,...,4.764544e-06,-0.000010,-7.738217e-06,-0.000003,0.000026,0.000014,2.506154e-06,-1.263259e-05,-0.000016,0.000014
21598,source_9,20211103-Run16,GR00004418,B42,JCP2022_040230,6703257770840297119,15,79.29,4.373112,1.899218,...,8.294126e-07,-0.000003,-9.144625e-06,-0.000005,0.000027,0.000018,-1.051333e-07,-9.165813e-06,-0.000016,0.000013
21599,source_9,20211103-Run16,GR00004421,B18,JCP2022_040230,6703257770840297119,15,79.29,4.373112,1.899218,...,-9.426994e-06,0.000002,-7.880708e-06,-0.000004,0.000009,0.000009,4.165263e-07,8.846787e-07,-0.000003,0.000005


In [100]:
print(f" Write to {VAL_OUTPUT_FILE}")

 Write to /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240829_1712_LAST_train_sub_val.csv


In [101]:
df_validation.to_csv(VAL_OUTPUT_FILE, mode = 'w', index = False, header = True, )

In [102]:
df_test = pd.read_csv(INPUT_FILE, header = 0,
                      skiprows = 298_800,
                      nrows = 12_600,
                      )
df_test.shape
df_test.info()
df_test.head(6)
df_test.tail(6)

(12600, 161)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12600 entries, 0 to 12599
Columns: 161 entries, source_9 to 4.881385393673554e-06
dtypes: float64(154), int64(2), object(5)
memory usage: 15.5+ MB


,source_9,20211103-Run16,GR00004421,B18,JCP2022_040230,6703257770840297119,15,79.29,4.373112017284307,1.899218417851373,...,-9.42699443839956e-06,2.0636507542803884e-06,-7.880707926233299e-06,-4.2592173485900275e-06,9.276894161303062e-06,8.631814125692472e-06,4.1652626237009827e-07,8.84678684087703e-07,-3.3064643503166735e-06,4.881385393673554e-06
0,source_9,20210914-Run9,GR00003302,H45,JCP2022_004021,6703456398488902534,6,58.04,4.061132,1.763727,...,-0.264010,-0.444511,-0.365870,-0.347954,0.059786,-0.007302,-0.337043,-0.304299,0.211058,0.238631
1,source_9,20210914-Run9,GR00003301,H21,JCP2022_004021,6703456398488902534,6,58.04,4.061132,1.763727,...,-0.251520,-0.437663,-0.371511,-0.335618,0.050832,0.003978,-0.318630,-0.293298,0.229234,0.212936
2,source_9,20210914-Run9,GR00003303,X45,JCP2022_004021,6703456398488902534,6,58.04,4.061132,1.763727,...,-0.204195,-0.326992,-0.292524,-0.264372,0.063065,-0.010498,-0.263440,-0.239843,0.161618,0.179531
3,source_3,CP_32_all_Phenix1,B40803aW,P21,JCP2022_102146,6704177726803547665,1,86.51,4.460260,1.937066,...,-0.000107,-0.001175,0.000746,-0.001319,0.001504,-0.001092,-0.000728,0.000044,-0.001463,-0.001287
4,source_6,p210928CPU2OS48hw384exp030JUMP,110000296383,P21,JCP2022_102146,6704177726803547665,1,86.51,4.460260,1.937066,...,-0.000194,-0.000748,0.000461,0.000389,0.001526,0.000013,-0.000557,0.000721,-0.000822,-0.000113
5,source_1,Batch5_20221030,UL001783,AE41,JCP2022_102146,6704177726803547665,1,86.51,4.460260,1.937066,...,0.000390,0.000246,0.001867,0.000457,0.001546,-0.001094,-0.000199,0.000035,-0.001469,0.000583


,source_9,20211103-Run16,GR00004421,B18,JCP2022_040230,6703257770840297119,15,79.29,4.373112017284307,1.899218417851373,...,-9.42699443839956e-06,2.0636507542803884e-06,-7.880707926233299e-06,-4.2592173485900275e-06,9.276894161303062e-06,8.631814125692472e-06,4.1652626237009827e-07,8.84678684087703e-07,-3.3064643503166735e-06,4.881385393673554e-06
12594,source_10,2021_08_20_U2OS_48_hr_run17,Dest210810-174829,P09,JCP2022_064379,7378624279819671723,11,49.85,3.909018,1.697665,...,0.000251,-0.000404,0.000284,0.000468,-0.000052,-0.000071,0.000475,-0.000520,-0.000103,0.000270
12595,source_1,Batch5_20221030,UL001791,AF11,JCP2022_064379,7378624279819671723,11,49.85,3.909018,1.697665,...,0.003074,-0.003054,0.002348,0.002050,0.000673,-0.001386,0.001569,-0.002407,0.004988,0.002304
12596,source_3,CP_32_all_Phenix1,B40703cW,P06,JCP2022_064379,7378624279819671723,11,49.85,3.909018,1.697665,...,0.002918,-0.002243,0.002225,0.002300,0.000392,-0.001115,0.001708,-0.002388,0.004796,0.002558
12597,source_10,2021_08_17_U2OS_48_hr_run16,Dest210809-141809,L22,JCP2022_012996,7378937281046818562,2,106.40,4.667206,2.026942,...,-0.000216,-0.000080,0.000266,-0.000978,0.000531,-0.000642,-0.000358,-0.000003,-0.001294,-0.000866
12598,source_1,Batch3_20221010,UL000091,AB44,JCP2022_012996,7378937281046818562,2,106.40,4.667206,2.026942,...,-0.000305,0.000304,-0.000296,0.000014,-0.000210,0.000290,0.000034,0.000128,-0.000470,-0.000141
12599,source_6,p210906CPU2OS48hw384exp025JUMP,110000294885,N22,JCP2022_012996,7378937281046818562,2,106.40,4.667206,2.026942,...,-0.000563,0.000515,0.000131,-0.000265,0.000556,-0.000432,-0.000587,-0.000313,-0.001424,0.000644


In [103]:
print(f" Write to {TEST_OUTPUT_FILE}")

 Write to /home/kevin/WSL-shared/cellpainting/cj-datasets/output_11102023/3_sample_embeddings/3smpl_prfl_embedding_161_HashOrder_SNNL_20240829_1712_LAST_train_sub_test.csv


In [104]:
df_test.to_csv(TEST_OUTPUT_FILE, mode = 'w', index = False, header = True, )

In [105]:
del df_validation, df_test